In [1]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import OnlinePDFLoader

In [2]:
#local_path = r"D:\GitHub\Projetos\Mestrado\EnergyContext\2599.pdf"
local_path = r"D:\GitHub\Projetos\Mestrado\EnergyContext\appendixa_0.pdf"

# Local PDF file uploads
if local_path:
  loader = UnstructuredPDFLoader(file_path=local_path)
  data = loader.load()
else:
  print("Upload a PDF file")

In [3]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [4]:
# Split and chunk 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
chunks = text_splitter.split_documents(data)

In [5]:
# Add to vector database
vector_db = Chroma.from_documents(
    documents=chunks, 
    embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True),
    collection_name="local-rag"
)

OllamaEmbeddings: 100%|██████████| 3/3 [00:08<00:00,  2.99s/it]


In [6]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [7]:
# LLM from Ollama
local_model = "mistral"
llm = ChatOllama(model=local_model)

In [8]:
'''QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)'''
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database, and only the database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [9]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [10]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [13]:
chain.invoke("What is the relation between Megawatt and Megawatt-hour?")

OllamaEmbeddings: 100%|██████████| 1/1 [00:04<00:00,  4.34s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.05s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.08s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.08s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.12s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


' A Megawatt (MW) is a unit of power, representing one million watts. It measures the rate at which energy is being produced or consumed per second. On the other hand, a Megawatt-hour (MWh) is a unit of energy, representing one million watt-hours. In simple terms, one MW-hour equals the energy consumed by using one MegaWatt for an hour. It quantifies the total energy consumption over a specific period of time, such as hours or days.'